In [44]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [183]:
dataset_filepath = './Dataset/zomato.csv'
df = pd.read_csv(dataset_filepath)
#df.head()

In [184]:
print(datetime.now())
# Remove commas from in between the digits
df['approx_cost(for two people)'] = df['approx_cost(for two people)'].str.replace(",","")

# Drop rows from the dataset with NaN in the "approx_cost(for two people)" column
df = df.dropna(subset=['approx_cost(for two people)'])

# Convert the "approx_cost(for two people)" column to int
df['approx_cost(for two people)'] = df['approx_cost(for two people)'].astype(int)
# Drop duplicate restaurant entries based on name and address column
df = df.drop_duplicates(subset=['name','address'], keep='first')

# Drop columns that will not be used for preprocessing
cols_to_drop = ['url','phone','reviews_list','menu_item','cuisines','online_order','book_table','rate','votes', 'dish_liked', 'listed_in(city)']
df = df.drop(cols_to_drop, axis=1)

# Detect nulls in columns and drop those rows
columns = df.columns.tolist()
print("Null/Nan in remaining columns:")
null_present = []
for c in columns:
    null_present.append(df[c].isnull().any())
    print(f"{c}: {df[c].isnull().any()}")
if(True in null_present):
    df = df.dropna()
    print("Dropped remaining rows with null values!")
else:
    print("No nulls values present!")

# Calculate mean cost per location i.e average cost of all restaurants in a given area
# Add a new column in df with means corresponding to the restaurant locations
mean_cost_location = df.groupby('location', as_index=False)['approx_cost(for two people)'].mean()
mean_cost_location.rename(columns={'approx_cost(for two people)':'mean_cost'}, inplace=True)
df['mean_cost_by_location'] = 0.0
locations = mean_cost_location['location'].tolist()
for location in locations:
    indices = df.index[df['location'] == location].tolist()
    mean_cost = mean_cost_location[mean_cost_location['location'] == location]['mean_cost'].tolist()[0]
    df['mean_cost_by_location'][indices] = mean_cost

# Calculate the deviation of each restaurant's cost for two from the corresponding mean_cost_location
df['delta_mean_cost_by_location'] = df['approx_cost(for two people)'] - df['mean_cost_by_location']

# Calculate mean cost per restaurant type i.e avg cost of restaurants per rest_type
mean_cost_rest_type = df.groupby('rest_type', as_index=False)['approx_cost(for two people)'].mean()
mean_cost_rest_type.rename(columns={'approx_cost(for two people)':'mean_cost'}, inplace=True)
df['mean_cost_by_rest_type'] = 0.0
restaurant_types = mean_cost_rest_type['rest_type'].tolist()
for rest_type in restaurant_types:
    indices = df.index[df['rest_type'] == rest_type].tolist()
    mean_cost = mean_cost_rest_type[mean_cost_rest_type['rest_type'] == rest_type]['mean_cost'].tolist()[0]
    df['mean_cost_by_rest_type'][indices] = mean_cost

# Calculate deviation between cost of the restaurant and corresponding mean_cost_rest_type
df['delta_mean_cost_by_rest_type'] = df['approx_cost(for two people)'] - df['mean_cost_by_rest_type']

# Calculate mean cost per rest_type, given the location
mean_rest_type_and_loc = df.groupby(['rest_type', 'location'], as_index=False)['approx_cost(for two people)'].mean()
mean_rest_type_and_loc.rename(columns={'approx_cost(for two people)':'mean_cost'}, inplace=True)
df['mean_cost_by_rest_type_and_loc'] = 0.0
for row in mean_rest_type_and_loc.itertuples(index=False):
    rest_type = row[0]
    location = row[1]
    mean_cost = row[2]
    indices = df.index[(df['rest_type']==rest_type) & (df['location']==location)].tolist()
    df['mean_cost_by_rest_type_and_loc'][indices] = mean_cost

# Calculate deviation between cost of the restaurant and corresponding mean_rest_type_and_location
df['delta_mean_cost_by_rest_type_and_loc'] = df['approx_cost(for two people)'] - df['mean_cost_by_rest_type_and_loc']

# Encode information about whether a restaurant is a chain or not.
# If yes, One-Hot-Encode which chain the restaurant belongs to.
# Else, the column "non_chain_restaurant" will have the value 1.0
restaurant_counts = df['name'].value_counts()
list_of_restaurants = df['name'].unique().tolist()
list_of_chains = []
for restaurant in list_of_restaurants:
    if restaurant_counts[restaurant] > 1:
        list_of_chains.append(restaurant)
        df[restaurant] = 0.0
df['non_chain_restaurant'] = 1.0

for restaurant in chains:
    indices = df.index[df['name'] == restaurant].tolist()
    df[restaurant][indices] = 1.0
    df['non_chain_restaurant'][indices] = 0.0

# Pending: One-Hot-Encode the "listed_in(type)" column

# Pending: Normalize all columns
print(datetime.now())

2019-06-23 22:13:35.318682
Null/Nan in remaining columns:
address: False
name: False
location: False
rest_type: True
approx_cost(for two people): False
listed_in(type): False
Dropped remaining rows with null values!
2019-06-23 22:17:20.111272


In [185]:
df.head()

,address,name,location,rest_type,approx_cost(for two people),listed_in(type),mean_cost_by_location,delta_mean_cost_by_location,mean_cost_by_rest_type,delta_mean_cost_by_rest_type,...,BBQ Nation Delivery,Durga Mess,Kake Ka Hotel,Kusum Rolls,RS Shiv Sagar Express,Samudra Restaurant,Lot Like Crepes & Cafe,Ghar ka Khana,New Shalimar Hotel,non_chain_restaurant
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Banashankari,Casual Dining,800,Buffet,380.101695,419.898305,753.526505,46.473495,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Banashankari,Casual Dining,800,Buffet,380.101695,419.898305,753.526505,46.473495,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Banashankari,"Cafe, Casual Dining",800,Buffet,380.101695,419.898305,846.000000,-46.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,Banashankari,Quick Bites,300,Buffet,380.101695,-80.101695,310.155626,-10.155626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,Basavanagudi,Casual Dining,600,Buffet,339.620853,260.379147,753.526505,-153.526505,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [186]:
df.shape

(12390, 1277)

In [187]:
df['approx_cost(for two people)'].unique()

array([ 800,  300,  600,  700,  550,  500,  450,  650,  400,  900,  200,
        750,  150,  850,  100, 1200,  350,  250,  950, 1000, 1500, 1300,
        199,   80, 1100,  160, 1600,  230,  130,   50,  190, 1700, 1400,
        180, 1350, 2200, 2000, 1800, 1900,  330, 2500, 2100, 3000, 2800,
       3400,   40, 1250, 3500, 4000, 2400, 2600,  120, 1450,  469,   70,
       3200,   60,  560,  240,  360, 6000, 1050, 2300, 4100, 5000, 3700,
       1650, 2700, 4500,  140])